In [ ]:

%%capture --no-stderr
%pip install --quiet -U langgraph langchain_community langchain_core tavily-python wikipedia
%pip install -qU langchain_google_genai

In [ ]:
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="LANGSMITH_API_KEY"
LANGSMITH_PROJECT="researchproject13"

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = LANGSMITH_TRACING
os.environ["LANGCHAIN_PROJECT"] = LANGSMITH_PROJECT
os.environ["LANGCHAIN_API_KEY"] = LANGSMITH_API_KEY

In [ ]:
import os
os.environ["TAVILY_API_KEY"] = "TAVILY_API_KEY"

In [ ]:
from operator import add
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.types import Send
from langgraph.graph import END, StateGraph, START
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display,Image,Markdown
from langchain_core.messages import HumanMessage,SystemMessage
from langgraph.graph import MessagesState
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AnyMessage,AIMessage
from langgraph.graph.message import add_messages
from langchain_community.document_loaders import WikipediaLoader
from operator import add
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",api_key='Gemeni_api_key'  )

############################################### Creating Analysts #######################################################
class Analyst(BaseModel):
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    analyst_name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst in the context of the topic.")
    description: str = Field(description="Description of the analyst focus, concerns, and motives.")

    @property
    def persona(self) -> str:
        return f"analyst_name: {self.analyst_name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

class Perspectives(BaseModel):
    analysts: list[Analyst] = Field(description="Comprehensive list of analysts with their roles and affiliations.", )

class OverallState(TypedDict):
    pass_human_feedback:str
    topic_name:str
    analysts: list[Analyst]
    max_analysts:int
    content: str
    conclusion:str 
    introduction:str
    generated_report:str
    no_of_questions:int
    sections: Annotated[list[str],add]

creating_analyst_prompt = """You are tasked with creating a set of AI analyst personas. Follow these instructions carefully:

1. First, review the research topic:
{topic}

2. Examine any editorial feedback that has been optionally provided to guide creation of the analysts:

{pass_human_feedback}

3. Determine the most interesting themes based upon documents and / or feedback above.

4. Pick the top {max_analysts} themes.

5. Assign one analyst to each theme."""

def create_analyst(state:OverallState):
    topic_name=state['topic_name']
    max_analysts=state['max_analysts']
    pass_human_feedback=state.get('pass_human_feedback','')
    prompt = creating_analyst_prompt.format(topic=topic_name,pass_human_feedback=pass_human_feedback,max_analysts=max_analysts)
    messages = [SystemMessage(content=prompt)] + [HumanMessage(content='Generate a set of analysts')]
    response = model.with_structured_output(Perspectives).invoke(messages)
 
    return {'analysts':response.analysts}


############################################### Decide Node wheather passing humanfeedback or empty #######################################################
class InterviewState(MessagesState):
  # messages:Annotated[list[AnyMessage],add_messages]
  analyst:Analyst
  wiki_context: str
  web_context: str
  no_of_questions:int
  interview:str
  sections: list[str]


def decide_node1(state:InterviewState):
    get_human_feedback = state.get('pass_human_feedback','')
    no_of_questions=state.get('no_of_questions',3)
   
    if get_human_feedback:
        return 'create_analyst'
    else:
        topic_name=state['topic_name']
        human_msg = [HumanMessage( content=f"So you said you were writing an article on {topic_name}?") ]
        for analyst in state["analysts"]:
            return [Send('sub_graph_node',{"analyst": analyst, "messages":human_msg,"no_of_questions":no_of_questions})]
 

############################################### Generating Questions #######################################################
generating_ques_inst = """You are an analyst tasked with interviewing an expert to learn about a specific topic.

Your goal is boil down to interesting and specific insights related to your topic.

1. Interesting: Insights that people will find surprising or non-obvious.

2. Specific: Insights that avoid generalities and include specific examples from the expert.

Here is your topic of focus and set of goals: {analyst_persona}

Begin by introducing yourself using a name that fits your persona, and then ask your question.

Continue to ask questions to drill down and refine your understanding of the topic.

When you are satisfied with your understanding, complete the interview with: "Thank you so much for your help!"

Remember to stay in character throughout your response, reflecting the persona and goals provided to you."""


def generate_question(state:InterviewState):
  analyst_persona=state['analyst'].persona
  messages = state['messages']
  prompt = generating_ques_inst.format(analyst_persona=analyst_persona)
  questions =model.invoke([SystemMessage(content=prompt,)]+messages)
  return {'messages':[questions]}


############################################### Search from Web #######################################################
search_web_instr = """You will be given a conversation between an analyst and an expert.

Your goal is to generate a well-structured query for use in retrieval and / or web-search related to the conversation.

First, analyze the full conversation.

Pay particular attention to the final question posed by the analyst.

Convert this final question into a well-structured web search query"""
class SearchQuery(BaseModel):
  query:str = Field(description = 'Search query for retrieval.')

def search_from_web(state:InterviewState):
  tavily_search = TavilySearchResults( max_results=2)
  structured_llm = model.with_structured_output(SearchQuery)
  messages = state['messages']
  searchQuerySchema = structured_llm.invoke( messages)
 

  search_docs = tavily_search.invoke(searchQuerySchema.query)

  formatted_search_docs = "\n\n---\n\n".join(
        [
            f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
            for doc in search_docs
        ]
    )
 
  return {"web_context": formatted_search_docs}

############################################### Search from Wikipedia #######################################################

def search_from_wikipedia(state:InterviewState):
    messages = state['messages'] 
    structured_llm = model.with_structured_output(SearchQuery)
    searchQuerySchema = structured_llm.invoke( messages)

    try:
        final_query = searchQuerySchema.query
        search_docs = WikipediaLoader(query=final_query,load_max_docs=4).load()

        if not search_docs: 
            return {"wiki_context": ""}
        formatted_search_docs = "\n\n---\n\n".join(
            [
                f'<Document source="{doc.metadata["source"]}" page="{doc.metadata.get("page", "")}"/>\n{doc.page_content}\n</Document>'
                for doc in search_docs
            ]
        )
     
        return {"wiki_context": formatted_search_docs }

    except Exception as e: 
        return {"wiki_context": ""}


############################################### Generating Answer #######################################################
answer_instructions = """You are an expert being interviewed by an analyst.

Here is analyst area of focus: {goal}.

You goal is to answer a question posed by the interviewer.

To answer question, use this context:

Web Search: {web_context}
Wikipedia Search: {wiki_context}

When answering questions, follow these guidelines:

1. Use only the information provided in the context.

2. Do not introduce external information or make assumptions beyond what is explicitly stated in the context.

3. The context contain sources at the topic of each individual document.

4. Include these sources your answer next to any relevant statements. For example, for source # 1 use [1].

5. List your sources in order at the bottom of your answer. [1] Source 1, [2] Source 2, etc

6. If the source is: <Document source="assistant/docs/llama3_1.pdf" page="7"/>' then just list:

[1] assistant/docs/llama3_1.pdf, page 7

And skip the addition of the brackets as well as the Document source preamble in your citation."""

def final_answer(state:InterviewState):
    messages=state['messages'][-1]
    wiki_context = state.get("wiki_context", "")
    prompt = answer_instructions.format(web_context=state["web_context"],wiki_context=state["wiki_context"],goal=state["analyst"].persona)
    answer = model.invoke([SystemMessage(content=prompt)]+[messages])
    answer.name='expert'
    return {'messages':[answer]}


############################################### search from web #######################################################
def decide_node2(state:InterviewState):
  messages = state['messages']
  no_of_questions=state['no_of_questions']
  print(f'no_of_questions{no_of_questions}')
  num_resp = len([m for m in messages if isinstance(m,AIMessage) and m.name=='expert'])
  if num_resp>=no_of_questions:
    return 'save_interview'

  if "Thank you so much for your help" in messages[-2].content:
        return 'save_interview'
  return 'generate_question'



############################################### save interview #######################################################
from langchain_core.messages import get_buffer_string
def save_interview(state:InterviewState):
    messages = state['messages'] 
    final_resp = get_buffer_string(messages) 
    return {'interview':final_resp}

############################################### write section #######################################################
write_section_instructions = """You are an expert technical writer. 
            
Your task is to create a short, easily digestible section of a report based on a set of source documents.

1. Analyze the content of the source documents: 
- The name of each source document is at the start of the document, with the <Document tag.
        
2. Create a report structure using markdown formatting:
- Use ## for the section title
- Use ### for sub-section headers
        
3. Write the report following this structure:
a. Title (## header)
b. Summary (### header)
c. Sources (### header)

4. Make your title engaging based upon the focus area of the analyst: 
{analyst_desc}

5. For the summary section:
- Set up summary with general background / context related to the focus area of the analyst
- Emphasize what is novel, interesting, or surprising about insights gathered from the interview
- Create a numbered list of source documents, as you use them
- Do not mention the names of interviewers or experts
- Aim for approximately 400 words maximum
- Use numbered sources in your report (e.g., [1], [2]) based on information from source documents
        
6. In the Sources section:
- Include all sources used in your report
- Provide full links to relevant websites or specific document paths
- Separate each source by a newline. Use two spaces at the end of each line to create a newline in Markdown.
- It will look like:

### Sources
[1] Link or Document name
[2] Link or Document name

7. Be sure to combine sources. For example this is not correct:

[3] https://ai.meta.com/blog/meta-llama-3-1/
[4] https://ai.meta.com/blog/meta-llama-3-1/

There should be no redundant sources. It should simply be:

[3] https://ai.meta.com/blog/meta-llama-3-1/
        
8. Final review:
- Ensure the report follows the required structure
- Include no preamble before the title of the report
- Check that all guidelines have been followed"""

def write_section(state:InterviewState): 
    web_context = state["web_context"]
    wike_context = state["wiki_context"]
    context = web_context + wike_context
    analyst_desc = state["analyst"].description
    system_message = write_section_instructions.format(analyst_desc=analyst_desc)
    section = model.invoke([SystemMessage(content=system_message)]+[HumanMessage(content=f"Use this source to write your section: {context}")])  
    return {"sections": [section.content]}

############################################### section introduction #######################################################
intro_conclusion_instructions = """You are a technical writer finishing a report on {topic_name}

You will be given all of the sections of the report.

You job is to write a crisp and compelling introduction or conclusion section.

The user will instruct you whether to write the introduction or conclusion.

Include no pre-amble for either section.

Target around 100 words, crisply previewing (for introduction) or recapping (for conclusion) all of the sections of the report.

Use markdown formatting. 

For your introduction, create a compelling title and use the # header for the title.

For your introduction, use ## Introduction as the section header. 

For your conclusion, use ## Conclusion as the section header.

Here are the sections to reflect on for writing: {formatted_str_sections}"""

def seaction_intro(state:OverallState):
    sections = state["sections"]
    topic_name = state["topic_name"] 
    formatted_str_sections = "\n\n".join([f"{section}" for section in sections]) 
    instructions = intro_conclusion_instructions.format(topic_name='topic_name', formatted_str_sections=formatted_str_sections)    
    intro = model.invoke([instructions]+[HumanMessage(content=f"Write the report introduction")]) 
    return {"introduction": intro.content}

############################################### section report #######################################################
report_instruction_prompt ="""You are a technical writer creating a report on this overall topic: 

{topic_name}
    
You have a team of analysts. Each analyst has done two things: 

1. They conducted an interview with an expert on a specific sub-topic.
2. They write up their finding into a memo.

Your task: 

1. You will be given a collection of memos from your analysts.
2. Think carefully about the insights from each memo.
3. Consolidate these into a crisp overall summary that ties together the central ideas from all of the memos. 
4. Summarize the central points in each memo into a cohesive single narrative.

To format your report:
 
1. Use markdown formatting. 
2. Include no pre-amble for the report.
3. Use no sub-heading. 
4. Start your report with a single title header: ## Insights
5. Do not mention any analyst names in your report.
6. Preserve any citations in the memos, which will be annotated in brackets, for example [1] or [2].
7. Create a final, consolidated list of sources and add to a Sources section with the `## Sources` header.
8. List your sources in order and do not repeat.

[1] Source 1
[2] Source 2

Here are the memos from your analysts to build your report from: 

{formatted_sections}"""

def section_report(state:OverallState):
    sections = state["sections"]
    topic_name = state["topic_name"] 
    formatted_sections = "\n\n".join([f"{section}" for section in sections]) 
    system_message = report_instruction_prompt.format(topic_name=topic_name, formatted_sections=formatted_sections)    
    report = model.invoke([SystemMessage(content=system_message)]+[HumanMessage(content=f"Write a report based upon these memos.")]) 
    return {"content": report.content}


############################################### section conculsion #######################################################
def section_conculsion(state:OverallState):
    sections = state["sections"]
    topic_name = state["topic_name"]
    formatted_str_sections = "\n\n".join([f"{section}" for section in sections])
    instructions = intro_conclusion_instructions.format(topic_name=topic_name, formatted_str_sections=formatted_str_sections)    
    conclusion = model.invoke([instructions]+[HumanMessage(content=f"Write the report conclusion")]) 
    return {"conclusion": conclusion.content}
############################################### final report #######################################################
def final_report(state:OverallState):
    content = state["content"]
    if content.startswith("## Insights"):
        content = content.strip("## Insights")
    if "## Sources" in content:
        try:
            content, sources = content.split("\n## Sources\n")
        except:
            sources = None
    else:
        sources = None

    final_report = state["introduction"] + "\n\n---\n\n" + content + "\n\n---\n\n" + state["conclusion"]
    if sources is not None:
        final_report += "\n\n## Sources\n" + sources
      
    return {"generated_report": final_report}

############################################### search from web #######################################################

def human_feedback(state:OverallState):
    pass




sub_graph = StateGraph(InterviewState)
sub_graph.add_node("generate_question", generate_question)
sub_graph.add_node('search_from_web',search_from_web)
sub_graph.add_node('search_from_wikipedia',search_from_wikipedia)
sub_graph.add_node('final_answer',final_answer)
sub_graph.add_node('save_interview', save_interview)
sub_graph.add_node('write_section',write_section)

sub_graph.add_edge(START, "generate_question")
sub_graph.add_edge('generate_question', "search_from_web")
sub_graph.add_edge('generate_question', "search_from_wikipedia")
sub_graph.add_edge('search_from_web', "final_answer")
sub_graph.add_edge('search_from_wikipedia', "final_answer")
sub_graph.add_conditional_edges('final_answer',decide_node2,['save_interview','generate_question'])
sub_graph.add_edge('save_interview','write_section')
sub_graph.add_edge('write_section',END)
compile_sub_graph = sub_graph.compile()

graph = StateGraph(OverallState)
graph.add_node("create_analyst", create_analyst)
graph.add_node("human_feedback", human_feedback)
graph.add_node('sub_graph_node', sub_graph.compile())
graph.add_node('seaction_intro',seaction_intro)
graph.add_node('section_report',section_report)
graph.add_node('section_conculsion',section_conculsion)
graph.add_node('final_report',final_report)

graph.add_edge(START, "create_analyst")
graph.add_edge('create_analyst', "human_feedback")
graph.add_conditional_edges("human_feedback", decide_node1, ["create_analyst", 'sub_graph_node'])
graph.add_edge('sub_graph_node','seaction_intro')
graph.add_edge('sub_graph_node','section_report')
graph.add_edge('sub_graph_node','section_conculsion')
graph.add_edge('sub_graph_node','seaction_intro')
graph.add_edge('sub_graph_node','section_report')
graph.add_edge('sub_graph_node','section_conculsion')
graph.add_edge(['seaction_intro','section_report','section_conculsion'],'final_report')
graph.add_edge('final_report',END)

app = graph.compile()
display(Image(app.get_graph(xray=1).draw_mermaid_png()))
query = {'topic_name':'what is agentic AI its importance and its application','max_analysts':4,'no_of_questions':1 }
resp=app.invoke(query) 
markdown_text = resp["generated_report"]   
display(Markdown(markdown_text))
 
